<a href="https://colab.research.google.com/github/isabellacadisco/Algorithms-for-Massive-Data-course/blob/main/LSH4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
# download dataset from kaggle and save it in drive


#import os
os.environ['KAGGLE_USERNAME'] = "isabellacadisco"
os.environ['KAGGLE_KEY'] = "847325691c2399e41dc9c0b78fbe499e"
!kaggle datasets download -d xhlulu/medal-emnlp

100% 6.81G/6.82G [01:30<00:00, 91.8MB/s]
100% 6.82G/6.82G [01:30<00:00, 81.1MB/s]


In [3]:
!unzip medal-emnlp.zip #unzip dataset

Archive:  medal-emnlp.zip
  inflating: full_data.csv           
  inflating: pretrain_subset/test.csv  
  inflating: pretrain_subset/train.csv  
  inflating: pretrain_subset/valid.csv  


In [4]:
# set up spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null #install java as VM
!wget -q https://dlcdn.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz #download Spark
!tar xf spark-3.4.0-bin-hadoop3.tgz #unzip the file
!pip install -q findspark

In [5]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"

In [6]:
import findspark
findspark.init("spark-3.4.0-bin-hadoop3")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate() #SPARK SESSION

In [7]:
import pyspark
type(spark)

pyspark.sql.session.SparkSession

In [8]:
file_path = './full_data.csv'

In [9]:
data_df = spark.read.csv(file_path, header=True)

In [10]:
from pyspark.sql.functions import monotonically_increasing_id

In [11]:
dataset = data_df.select('TEXT').withColumn('doc_id', monotonically_increasing_id())

In [12]:
dataset.show()

+--------------------+------+
|                TEXT|doc_id|
+--------------------+------+
|alphabisabolol ha...|     0|
|a report is given...|     1|
|the virostatic co...|     2|
|rmi rmi and rmi a...|     3|
|a doubleblind stu...|     4|
|stroma from eithe...|     5|
|the effect of the...|     6|
|in one experiment...|     7|
|the presence of a...|     8|
|the reaction of g...|     9|
|choline acetyltra...|    10|
|increasing concen...|    11|
|the properties of...|    12|
|primary amines re...|    13|
|a purification pr...|    14|
|dihydrofolate red...|    15|
|ionization effect...|    16|
|kinetic analyses ...|    17|
|the nearultraviol...|    18|
|the circular pola...|    19|
+--------------------+------+
only showing top 20 rows



In [13]:
small_dataset = dataset.sample(withReplacement=False, fraction=0.000007, seed=42)

In [14]:
small_dataset.count()

94

In [15]:
small_dataset.show()

+--------------------+------------+
|                TEXT|      doc_id|
+--------------------+------------+
|a recent outbreak...|      106539|
|wefast gradientec...| 25769866227|
|this paper deals ...| 42949764250|
|there is a close ...| 51539638497|
|occasional and un...| 51539642085|
|the relative perc...| 77309421895|
|mixed alloys for ...|103079248410|
|a high incidence ...|103079293994|
|a housetohouse su...|103079331152|
|to characterize t...|120259124985|
|we studied the ef...|120259129263|
|the effects of cl...|137439020479|
|lipopolysaccharid...|154618911990|
|this case report ...|154618921437|
|a study of one of...|163208873442|
|rat fetuses at da...|188978671675|
|the catenins are ...|206158468968|
|when tissues are ...|214748392950|
|SS concentrations...|214748459668|
|the sensitivity o...|223338408901|
+--------------------+------------+
only showing top 20 rows



In [16]:
# TEXT PROCESSOR
!pip install spacy
!python -m spacy download en_core_web_sm



2023-09-11 21:37:29.960582: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-11 21:37:32.039187: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 41.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [17]:
# Import spaCy and download the English model
import spacy
import en_core_web_sm


In [18]:
# Load the spaCy English model FORSE NON SERVE BOH
nlp = en_core_web_sm.load()

In [19]:
class TextPreprocessor:
    def __init__(self):
        # Initialize spaCy resources
        self.nlp = spacy.load("en_core_web_sm")

    def preprocess(self, document):
        # Apply spaCy NLP pipeline
        doc = self.nlp(document)

        # Lemmatize and remove stopwords
        tokens = [token.lemma_ for token in doc if not token.is_stop]

        # Join tokens back into a text
        cleaned_text = " ".join(tokens)

        return cleaned_text

In [43]:
class TextPreprocessor_map:
    def __init__(self, dataframe):
        # Initialize spaCy resources
        self.nlp = spacy.load("en_core_web_sm")

        doc_id = dataframe[1]
        text = dataframe[0]

    def preprocess(self, dataframe):
        # Apply spaCy NLP pipeline
        doc = self.nlp(text)

        # Lemmatize and remove stopwords
        tokens = [token.lemma_ for token in doc if not token.is_stop]

        # Join tokens back into a text
        cleaned_text = " ".join(tokens)

        return (text, doc)

In [47]:
data_rdd = small_dataset.rdd

In [44]:
map_prep = TextPreprocessor_map

In [48]:
cleaned_data = data_rdd.map(map_prep)

In [49]:
cleaned_data.toDF().show()

TypeError: ignored

In [20]:
text_preprocessor_spacy = TextPreprocessor()

In [63]:
import string

In [64]:
def preprocess_text(document):
    # Initialize spaCy resources
    nlp = spacy.load("en_core_web_sm")

    # Apply spaCy NLP pipeline
    doc = nlp(document)

    # Lemmatize and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and token.text not in string.punctuation]

    # Join tokens back into a text
    cleaned_text = " ".join(tokens)

    return cleaned_text

In [65]:
# UDF --

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [66]:
# Register the UDF with Spark
preprocess_udf_spark = udf(preprocess_text, StringType())

In [67]:
df_prep = small_dataset.withColumn("preprocessed_text", preprocess_udf_spark(small_dataset['TEXT']))

In [68]:
df_prep.show()

+--------------------+------------+--------------------+
|                TEXT|      doc_id|   preprocessed_text|
+--------------------+------------+--------------------+
|a recent outbreak...|      106539|recent outbreak s...|
|wefast gradientec...| 25769866227|wefast gradientec...|
|this paper deals ...| 42949764250|paper deal patien...|
|there is a close ...| 51539638497|close association...|
|occasional and un...| 51539642085|occasional unpred...|
|the relative perc...| 77309421895|relative percent ...|
|mixed alloys for ...|103079248410|mix alloy dental ...|
|a high incidence ...|103079293994|high incidence go...|
|a housetohouse su...|103079331152|housetohouse surv...|
|to characterize t...|120259124985|characterize cont...|
|we studied the ef...|120259129263|study efficacy ca...|
|the effects of cl...|137439020479|effect clonidine ...|
|lipopolysaccharid...|154618911990|lipopolysaccharid...|
|this case report ...|154618921437|case report yearo...|
|a study of one of...|163208873

In [21]:
# HASH FAMILY


# ci sono degli import che non ho eseguito ma ho potuto comunque creare classe
# forse sono già in colab boh

import re, hashlib, math, time
from random import randint, seed

# però sono abbastanza certa che il seed mi serva

seed(16)

# HO MESSO BYTE RITORNO A 4, PENSO 2^32 BUCKET THEN
# BUT WHO KNOWS

class hashFamily:
    def __init__(self, i):
        self.resultSize = 4 # how many bytes we want back
        self.maxLen = 20 # how long can our i be (in decimal)
        self.salt = str(i).zfill(self.maxLen)[-self.maxLen:]
        self.id = i

    def get_hash_value(self, el_to_hash):
        return int(hashlib.sha1(str(el_to_hash).encode('utf-8') + self.salt.encode('utf-8')).hexdigest()[-self.resultSize:], 16)


In [22]:
# SHINGLER


class shingler:
    def __init__(self, k):

        if k > 0:
            self.k = int(k)
        else:
            self.k = 10

    #inner class utility
    #def process_doc(self, document):
     #   return re.sub("( )+|(\n)+"," ",document).lower()

    def get_shingles(self, document):
        shingles = set()

        #spacy_proc = TextPreprocessor()

        document= text_preprocessor_spacy.preprocess(document)

        for i in range(0, len(document)-self.k+1 ):
            shingles.add(document[i:i+self.k])
        return shingles

    def get_k(self):
        return self.k

    #return sorted hash
    def get_hashed_shingles(self, shingles_set):
        hash_function = hashFamily(0)
        return sorted( {hash_function.get_hash_value(s) for s in shingles_set} )

In [23]:
def shingling_map(row):
    out = []
    sh_instance = shingler(10)
    hashed_shingles = sh_instance.get_hashed_shingles( sh_instance.get_shingles( row['TEXT'] ) )
    signature_size = 50
    for i in range(0,signature_size): #signature size
        out.append( (row['doc_id'], hashed_shingles, i ) )
    #return an iterator to use flatMap => produce more than one key-value pair as output (namely one per hash function)
    return iter(out)




In [24]:
rdd_0 = small_dataset.rdd.flatMap(shingling_map)

In [25]:
rdd_0_df = rdd_0.toDF()

In [26]:
rdd_0_df.show()

+------+--------------------+---+
|    _1|                  _2| _3|
+------+--------------------+---+
|106539|[76, 297, 337, 51...|  0|
|106539|[76, 297, 337, 51...|  1|
|106539|[76, 297, 337, 51...|  2|
|106539|[76, 297, 337, 51...|  3|
|106539|[76, 297, 337, 51...|  4|
|106539|[76, 297, 337, 51...|  5|
|106539|[76, 297, 337, 51...|  6|
|106539|[76, 297, 337, 51...|  7|
|106539|[76, 297, 337, 51...|  8|
|106539|[76, 297, 337, 51...|  9|
|106539|[76, 297, 337, 51...| 10|
|106539|[76, 297, 337, 51...| 11|
|106539|[76, 297, 337, 51...| 12|
|106539|[76, 297, 337, 51...| 13|
|106539|[76, 297, 337, 51...| 14|
|106539|[76, 297, 337, 51...| 15|
|106539|[76, 297, 337, 51...| 16|
|106539|[76, 297, 337, 51...| 17|
|106539|[76, 297, 337, 51...| 18|
|106539|[76, 297, 337, 51...| 19|
+------+--------------------+---+
only showing top 20 rows



In [27]:
rdd_0_df.count()

4700

che è esattamente 94 (sample size) * 50 (sig matrix size) BELLA

## TO DO:
- CAPIRE COME E DOVE E QUANDO AGGIUNGERE CALCOLO LUNGHEZZA MEDIA IN CARATTERI DI TUTTI I DOC PER SCELTA OTTIMALE DI K
- SCELTA SIZE SIGNATURE MATRIX

- SIMILAR PAIRS DETECTION IMPLEMENTAZIONE CORRETTA
- SCELTA NUMERO BAND AND ROWS FOR LSH

- AMPLIFICATION AND OR???